In [118]:
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

seed = 7
numpy.random.seed(seed)

In [119]:
arrest_dataframe = pd.read_csv('BPD_Arrests_sanitized.csv')
print len(arrest_dataframe)

45092


In [120]:
arrest_dataframe.head(2)

,Unnamed: 0,Arrest,Age,Sex,Race,ArrestDate,ArrestTime,ArrestLocation,IncidentOffense,IncidentLocation,Charge,ChargeDescription,District,Neighborhood,ArrestDay,NormalizedArrestDate,ArrestHour
0,1,16147119.0,33.0,M,B,10/15/2016,23:00,100 W NORTH AVE,4C,W NORTH AV & N CHARLES ST,1 1415,DOMESTIC ASSAULT,Central,Charles North,Saturday,2016-10-15,23
1,2,16147109.0,27.0,M,B,10/15/2016,22:45,2300 E JEFFERSON ST,4C,2300 JEFFERSON ST,1 1420,ASSAULT,Southeastern,McElderry Park,Saturday,2016-10-15,22


In [47]:
'''
print 'len(arrest_dataframe)', len(arrest_dataframe)
print 'Arrest :', len(arrest_dataframe['Arrest'].unique())
print 'Age', len(arrest_dataframe['Age'].unique())
print 'Sex', len(arrest_dataframe['Sex'].unique())
print 'Race', len(arrest_dataframe['Race'].unique())
print 'ArrestDate', len(arrest_dataframe['ArrestDate'].unique())
print 'ArrestTime', len(arrest_dataframe['ArrestTime'].unique())
print 'ArrestLocation [Many Unknowns]', len(arrest_dataframe['ArrestLocation'].unique())
print 'IncidentOffense [Many Unknowns]', len(arrest_dataframe['IncidentOffense'].unique())
print 'IncidentLocation [Many Unknowns]', len(arrest_dataframe['IncidentLocation'].unique())
print 'Charge', len(arrest_dataframe['Charge'].unique())
print 'ChargeDescription', len(arrest_dataframe['ChargeDescription'].unique())
print 'District', len(arrest_dataframe['District'].unique())
print 'Neighborhood', len(arrest_dataframe['Neighborhood'].unique())
'''
print 'hi'

hi


In [121]:
Z = arrest_dataframe.sort_values(['IncidentOffense'],axis=0, ascending=True)
#Y = Z['IncidentOffense']
Y = arrest_dataframe['IncidentOffense']

#A = set()
#for i in range(1,len(Y)):
#    A.add(Y[i][0])
#print (A) #set(['1', '3', '2', '5', '4', '7', '6', '9', '8', 'U'])

TempValue = Y[Y.str[0] == '1']
print len(TempValue)
print TempValue.unique()

#print len(Y[Y == '104-Malicious Burning'])
#print (Y[Y.str.contains('108-Liquor')])
#print len(Y[Y == 'UNKNOWN OFFENSE'])

'''
num = '9'
Y = Y.apply(lambda offns: num if (offns[0] == num) else offns)
CrimeOne = Y[Y == num]
print len(CrimeOne)

for i in range(1,10):
    CrimeOne = Y[Y == str(i)]
    print len(CrimeOne)    
'''

1706
['118' '111' '119' '115' '1A' '117' '1B' '116' '112' '108' '114' '106'
 '103' '113' '109' '102' '107' '105' '110' '104']


"\nnum = '9'\nY = Y.apply(lambda offns: num if (offns[0] == num) else offns)\nCrimeOne = Y[Y == num]\nprint len(CrimeOne)\n\nfor i in range(1,10):\n    CrimeOne = Y[Y == str(i)]\n    print len(CrimeOne)    \n"

In [126]:
#X = pd.concat([arrest_dataframe['Age'], pd.get_dummies(arrest_dataframe['Sex'], prefix = 'S'), pd.get_dummies(arrest_dataframe['Race'], prefix = 'R')], axis=1)
#Y = arrest_dataframe['IncidentOffense']

X = pd.concat([pd.get_dummies(arrest_dataframe['IncidentOffense'], prefix = 'offns')], axis=1)
Y = arrest_dataframe['Age']

#X = pd.concat([pd.get_dummies(arrest_dataframe['Age'], prefix = 'A')], axis=1)
#Y = arrest_dataframe['IncidentOffense']

print 'len(X) = ', len(X), 'len(Y) = ', len(Y)

X_cols = len(X.columns)
print X_cols
print X.columns

print len(Y.unique())
print Y.unique()

len(X) =  45092 len(Y) =  45092
142
Index([u'offns_102', u'offns_103', u'offns_104', u'offns_105', u'offns_106',
       u'offns_107', u'offns_108', u'offns_109', u'offns_110', u'offns_111',
       ...
       u'offns_8H', u'offns_8J', u'offns_93', u'offns_94', u'offns_95',
       u'offns_96', u'offns_96A', u'offns_96B', u'offns_97', u'offns_98'],
      dtype='object', length=142)
69
[ 33.  27.  24.  41.  32.  25.  58.  59.  34.  39.  45.  31.  49.  26.  20.
  22.  47.  19.  23.  30.  57.  18.  29.  36.  21.  28.  50.  35.  53.  37.
  46.  54.  44.  43.  55.  38.  61.  40.  52.  56.  42.  63.  60.  51.  48.
  64.  65.  69.  17.   0.  16.  15.  62.  67.  66.  71.  68.  75.  70.  14.
  72.  74.  80.  76.  73.  81.  87.  77.  89.]


In [127]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [128]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X,Y)
model.score(X,Y)

0.0047014991572784529

In [129]:
model = Sequential()

model.add(Dense(X_cols, input_dim=X_cols, init='uniform', activation='relu'))
model.add(Dense(X_cols, init='uniform', activation='relu'))
model.add(Dense(len(Y.unique()), init='uniform', activation='sigmoid'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit model
model.fit(X.values, dummy_y, nb_epoch=50, batch_size=10, shuffle=True, verbose=1)

# Evaluate model
model.evaluate(X.values, dummy_y, batch_size=5, verbose=1)

Epoch 1/50
16490/45092 [=========>....................] - ETA: 7s - loss: 3.7127 - acc: 0.0421

KeyboardInterrupt: 